In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
import utils

In [3]:
itp_id = 300

In [4]:
pd.set_option("display.max_columns", 100)


In [5]:
analysis_date_start = '2022-05-01'
analysis_date_end = '2022-05-31'

In [21]:
rt_sched = query_sql(
    """
SELECT *
FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_sched_ran`
"""
)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [24]:
rt_sched.head()

,calitp_itp_id,calitp_url_number,service_date,num_sched,num_rt,pct_ran
0,300,0,2022-05-22,770,543,0.71
1,300,0,2022-05-01,771,527,0.68
2,300,0,2022-05-15,771,527,0.68
3,300,0,2022-05-29,773,559,0.72
4,300,0,2022-05-08,773,504,0.65


In [52]:
rt_sched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   calitp_itp_id      31 non-null     int64         
 1   calitp_url_number  31 non-null     int64         
 2   service_date       31 non-null     datetime64[ns]
 3   num_sched          31 non-null     int64         
 4   num_rt             31 non-null     int64         
 5   pct_ran            31 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 1.6 KB


In [53]:
rt_sched['service_date'] = pd.to_datetime(rt_sched['service_date'])

In [55]:
rt_sched['weekday'] = pd.Series(rt_sched.service_date).dt.day_name()

## Using Functions (without DBT tables)

In [6]:
#gtfs_daily = utils.load_schedule_data(analysis_date_start, analysis_date_end, itp_id)

In [7]:
#gtfs_daily

In [8]:
#gtfs_daily.service_date.min()

In [9]:
#gtfs_daily.service_date.max()

In [10]:
#rt = utils.load_rt_data(analysis_date_start, analysis_date_end)

In [11]:
#rt

In [12]:
#rt['str_len'] = rt.trip_id.str.len()

In [13]:
#rt.str_len.value_counts()

In [14]:
#len(rt)

In [15]:
#rt.trip_id.nunique()

In [16]:
#len(gtfs_daily)

In [17]:
#gtfs_daily.trip_id.nunique()

In [7]:
#date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [8]:

# from datetime import date, timedelta
# start_date = date(2022, 5, 1)
# end_date = date(2022, 5, 31)

In [9]:
# daterange = pd.date_range(start_date, end_date)

In [10]:
# (daterange)

In [11]:
#above range does not work with function

In [12]:
# dates = set()

# def daterange2(start, end):
#     for n in range(int((end - start).days) + 1):
#         yield start + timedelta(n)

# for single_date in daterange2(start_date, end_date):
#     dates.add(single_date.strftime('%Y-%m-%d'))

In [13]:
# (dates)

In [14]:
# rt['date'] = pd.to_datetime(rt['date'])

In [15]:
# gtfs_daily['service_date'] = pd.to_datetime(gtfs_daily['service_date'])

In [16]:
# single_date2 = '2022-05-08'

# (rt>>filter(_.date == single_date2))


In [17]:
# def get_pct_ran_df2(itp_id, list_of_dates):
#     pcts = []
#     for single_date in list_of_dates:
#         gtfs_daily2 = (gtfs_daily>>filter(_.service_date == single_date))
#         rt2 = (rt>>filter(_.date == single_date))
#         sched_rt_df = (pd.merge(gtfs_daily2, rt2, how='outer', on='trip_id', indicator='have_rt'))
#         #pct_ran = (utils.get_pct_ran(sched_rt_df, single_date))
#         day_pct_ran = {}
#         day_pct_ran['date'] = single_date
#         if ((len(sched_rt_df))!=0):
#             day_pct_ran['pct_trips_ran'] = ((len(sched_rt_df>>filter(_.have_rt=='both')))/(len(gtfs_daily2)))
#         elif ((len(sched_rt_df))==0):
#             day_pct_ran['pct_trips_ran'] = ''
#         pct_ran = pd.DataFrame([day_pct_ran])
#         pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
#         pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
#         pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
#         pct_ran['nunique_sched'] = (gtfs_daily2.trip_id.nunique())
#         pct_ran['nunique_rt'] = (rt2.trip_id.nunique())

#         pcts.append(pct_ran)                                                    
#         #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
#     pcts = pd.concat(pcts)
#     pcts = pcts>>arrange(_.date)
#     return pd.DataFrame(pcts)


In [18]:
# may = utils.get_pct_ran_df(itp_id, dates, gtfs_daily, rt)

In [19]:
# may

## Charting

In [20]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [23]:
rt_sched.sample()

,calitp_itp_id,calitp_url_number,service_date,num_sched,num_rt,pct_ran
5,300,0,2022-05-13,1560,1439,0.92


In [59]:
bar = (alt.Chart(rt_sched)
        .mark_bar()
        .encode(
            x=alt.X('service_date', title=_dla_utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_ran', title=_dla_utils.labeling('pct_ran')),
            color=alt.Color(
                'pct_ran',
                scale=alt.Scale(range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                legend=alt.Legend(title=(_dla_utils.labeling('pct_ran')), symbolLimit=10)
            )
        ).properties(title='"Percent of Scheduled Trips with RT Data"'))
    
chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)

In [60]:
df_long =  (rt_sched>>select(_.service_date,
                         _.num_sched,
                         _.num_rt
                        ) 
             >> gather('measure',
                       'value',
                       _.num_sched,
                       _.num_rt
                      )
            )

In [61]:
df_long

,service_date,measure,value
0,2022-05-22,num_sched,770
1,2022-05-01,num_sched,771
2,2022-05-15,num_sched,771
3,2022-05-29,num_sched,773
4,2022-05-08,num_sched,773
...,...,...,...
57,2022-05-31,num_rt,1067
58,2022-05-16,num_rt,1030
59,2022-05-02,num_rt,1069
60,2022-05-09,num_rt,1060


In [97]:
bar = alt.Chart(df_long).mark_bar(opacity=1).encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('value', stack=None,  title=_dla_utils.labeling('number of trips')),
    color="measure",
)

chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)

In [63]:
rt_sched.sample()

,calitp_itp_id,calitp_url_number,service_date,num_sched,num_rt,pct_ran,weekday
6,300,0,2022-05-06,1561,1392,0.89,Friday


In [74]:
week = rt_sched>>group_by(_.weekday)>>summarize(avg_pct_ran = _.pct_ran.mean())

In [80]:
week

,weekday,avg_pct_ran
0,Friday,0.91
1,Monday,0.75
2,Saturday,0.71
3,Sunday,0.69
4,Thursday,0.91
5,Tuesday,0.87
6,Wednesday,0.91


In [82]:
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
week['weekday'] = pd.Categorical(week['weekday'], categories=cats, ordered=True)
week=week.sort_values('weekday')          

In [85]:
week

,weekday,avg_pct_ran
1,Monday,0.75
5,Tuesday,0.87
6,Wednesday,0.91
4,Thursday,0.91
0,Friday,0.91
2,Saturday,0.71
3,Sunday,0.69


In [91]:
bar = alt.Chart(week).mark_bar().encode(
    x=alt.X('weekday', title=_dla_utils.labeling('weekday'), sort=cats),
    y=alt.Y('avg_pct_ran', title=_dla_utils.labeling('avg_pct_ran')),
    color="weekday")
chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)